In [1]:
import cv2 
import numpy as np
import os
import numpy as np
from datetime import datetime

In [2]:
output_folder = 'C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/HappyScenario'
os.makedirs(output_folder, exist_ok=True)
print(f'output_folder: {output_folder}')

output_folder: C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/HappyScenario


In [4]:
img = cv2.imread('data\sample.jpg')
img_gray = cv2.imread('data\sample.jpg', cv2.IMREAD_GRAYSCALE)
template = cv2.imread(r'data\test.jpg', cv2.IMREAD_GRAYSCALE)

print(f'image shape: {img_gray.shape}')
print(f'template shape: {template.shape}')

image shape: (9000, 12600)
template shape: (61, 48)


In [5]:
# Store width and height of template in w and h 
w, h = template.shape[::-1] 

In [ ]:
# Define scale and threshold
scale = 0.9
threshold = 0.799

# Resize the template according to scale
template_resized = cv2.resize(template, (int(w * scale), int(h * scale)))
w_resized, h_resized = template_resized.shape[::-1]

# Detect edges in the grayscale image and apply template matching
result = cv2.matchTemplate(img_gray, template_resized, cv2.TM_CCOEFF_NORMED)

# Set threshold for detection
loc = np.where(result >= threshold)

# Prepare a list for bounding boxes
boxes = []
for pt in zip(*loc[::-1]):  # Switch columns and rows
    boxes.append([pt[0], pt[1], pt[0] + w_resized, pt[1] + h_resized])  # [x1, y1, x2, y2]

# Convert to numpy array for easier processing
boxes = np.array(boxes)

# Create an image to draw the detections
img_detected = img.copy()  # Create a copy of the original image to draw on

# Count detected items without NMS
count_detected = 0
detected_points = set()  # To track unique detections

# Draw rectangles around detected matches with red color
for box in boxes:
    # Create a tuple for the box coordinates
    box_tuple = (box[0], box[1], box[2], box[3])
    
    # Check if this box overlaps with previously detected boxes
    if not any((box[0] < x2 and box[2] > x1 and box[1] < y2 and box[3] > y1) for (x1, y1, x2, y2) in detected_points):
        cv2.rectangle(img_detected, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)  # Red color
        detected_points.add(box_tuple)  # Add this box to the set of detected points
        count_detected += 1  # Increment the count for each unique detection

# Create a filename based on scale and threshold
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"detected_scale_{scale}_threshold_{threshold}_{current_time}.jpg"
filepath = os.path.join(output_folder, filename)

# Save the detected image
cv2.imwrite(filepath, img_detected)

# Print the count of detected items
print(f'File Stored at: {filepath}\nfile_name: {filename}')
print(f"Scale: {scale}, Threshold: {threshold}, Count Detected: {count_detected}")

# Optionally display the last detected image
# cv2.imshow("Detected", img_detected) 
# cv2.waitKey(0)
# cv2.destroyAllWindows()

File Stored at: C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/HappyScenario\detected_scale_0.9_threshold_0.799_20241122_143814.jpg
 file_name: detected_scale_0.9_threshold_0.799_20241122_143814.jpg
Scale: 0.9, Threshold: 0.799, Count Detected: 91
